## 이명박 데이터 수집

In [7]:
import re
import time
import urllib
from bs4 import BeautifulSoup
import html
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import json
import numpy as np

#모듈 불러오기

In [2]:
def remove_tag(my_str):
    ## 태그를 지우는 함수
    p = re.compile('(<([^>]+)>)')
    return p.sub('', my_str)

In [3]:
client_id =
client_secret = 

#네이버 뉴스에서 데이터를 수집
base_url = 'https://openapi.naver.com/v1/search/news.json'

#여러 개의 쿼리를 주어 한번에 여러개의 데이터를 수집
queries = [
    '"李대통령" +2008', '"李대통령" +2009','"李대통령" +2010','"李대통령" +2011','"李대통령" +2012',
    '"이대통령" +2008', '"이대통령" +2009', '"이대통령" +2010', '"이대통령" +2011','"이대통령" +2012'
]
n_display = 100

#정확도 순으로 탐색
sort = 'sim'

In [4]:
# 네이버 API 설정
all_links = []
all_pubdates = []

# HTML 태그 제거 함수
def remove_tag(text):
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

# 반복문을 사용해 주제를 반복해 수집하도록 설정
for query in queries:
    encQuery = urllib.parse.quote(query)
    for start in range(1, 1001, 100):
        url = f'{base_url}?query={encQuery}&display={n_display}&start={start}&sort={sort}'
        my_request = urllib.request.Request(url)
        my_request.add_header("X-Naver-Client-Id", client_id)
        my_request.add_header("X-Naver-Client-Secret", client_secret)
        
        try:
            response = urllib.request.urlopen(my_request)
            rescode = response.getcode()
            if rescode == 200:
                response_body = response.read()
            else:
                print("Error Code:" + rescode)
                continue
        except urllib.error.HTTPError as e:
            print(f"HTTP Error: {e.code} - {e.reason}")
            continue
        except urllib.error.URLError as e:
            print(f"URL Error: {e.reason}")
            continue

        search_results = response_body.decode('utf-8')
        search_results = json.loads(search_results)
        
        if 'items' not in search_results:
            print("No items in search results")
            continue

        p = re.compile('n.news.naver.com')
        for item in search_results['items']:
            if p.search(item['link']) is not None:
                title = html.unescape(remove_tag(item['title']))
                pubdate = item['pubDate']
                link = item['link'].replace('\\', '')
                all_links.append(link)
                all_pubdates.append(pubdate)
        
        time.sleep(1)  # API 호출 제한 방지를 위한 대기 시간

df_link = pd.DataFrame({'link': all_links, 'pubdate': all_pubdates})

In [5]:
df_link

,link,pubdate
0,https://n.news.naver.com/mnews/article/001/001...,"Fri, 31 May 2024 13:00:00 +0900"
1,https://n.news.naver.com/mnews/article/023/000...,"Fri, 31 May 2024 15:55:00 +0900"
2,https://n.news.naver.com/mnews/article/366/000...,"Thu, 30 May 2024 15:50:00 +0900"
3,https://n.news.naver.com/mnews/article/629/000...,"Thu, 30 May 2024 16:43:00 +0900"
4,https://n.news.naver.com/mnews/article/277/000...,"Thu, 30 May 2024 16:10:00 +0900"
...,...,...
3502,https://n.news.naver.com/mnews/article/018/000...,"Wed, 01 May 2024 16:09:00 +0900"
3503,https://n.news.naver.com/mnews/article/422/000...,"Wed, 01 May 2024 20:45:00 +0900"
3504,https://n.news.naver.com/mnews/article/215/000...,"Mon, 06 May 2024 16:18:00 +0900"
3505,https://n.news.naver.com/mnews/article/055/000...,"Wed, 01 May 2024 06:35:00 +0900"


In [6]:
# Selenium을 사용하여 각 링크에서 데이터 수집
driver = webdriver.Chrome()
driver.implicitly_wait(3)

In [7]:
results = []

# df_link의 각 URL에 대해 반복 실행
for i, url in enumerate(df_link['link']):
    driver.get(url)
    
    # 제목을 얻기
    try:
        title = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="title_area"]/span'))
        ).text
    except Exception as e:
        print(f"Error fetching title for URL {url}: {e}")
        title = ""

    # 본문을 얻기
    try:
        content = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="newsct_article"]'))
        ).text
    except Exception as e:
        print(f"Error fetching content for URL {url}: {e}")
        content = ""

    # 언론사 정보를 얻기
    try:
        copyright_text = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, '//p[@class="c_text"]'))
        ).text
        if "ⓒ" in copyright_text:
            agency = copyright_text.split("ⓒ")[1].strip()
        else:
            agency = ""
    except Exception as e:
        print(f"Error fetching agency for URL {url}: {e}")
        agency = ""
    
    # 댓글을 얻기
    try:
        # 전체 댓글 보기 버튼을 클릭
        all_comment_btn = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.CLASS_NAME, 'u_cbox_in_view_comment'))
        )
        all_comment_btn.click()
        
        comments = []
        while True:
            try:
                # 더 많은 댓글 보기 버튼을 클릭
                more_comment_btn = WebDriverWait(driver, 5).until(
                    EC.element_to_be_clickable((By.CLASS_NAME, 'u_cbox_btn_more'))
                )
                more_comment_btn.click()
                time.sleep(1)
            except:
                break
        
        # 댓글 요소를 찾고 텍스트를 저장
        comments_elements = driver.find_elements(By.CLASS_NAME, 'u_cbox_contents')
        comments = [x.text for x in comments_elements]
    except Exception as e:
        print(f"Error fetching comments for URL {url}: {e}")
        comments = []

    time.sleep(1)
    results.append([url, title, agency, content, comments])

# 드라이버 종료
driver.quit()

Error fetching comments for URL https://n.news.naver.com/mnews/article/449/0000276415?sid=115: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/366/0000995860?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/023/0003837786?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/029/0002876031?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/029/0002876236?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/008/0005043158?sid=101: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/052/0002039804?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/123/0002334901?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/029/0002876538?sid=101: Message: 

Error fetching comments for URL https://n.news

Error fetching comments for URL https://n.news.naver.com/mnews/article/016/0002249893?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/018/0005651988?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/005/0001665452?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/029/0002847140?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/087/0001017491?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/005/0001663750?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/366/0000959049?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/029/0002846763?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/029/0002846678?sid=100: Message: 

Error fetching comments for URL https://n.news

Error fetching comments for URL https://n.news.naver.com/mnews/article/052/0001953885?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/025/0003310069?sid=102: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/029/0002833563?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/001/0014479365?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/029/0002832253?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/421/0007149881?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/421/0007149876?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/421/0007149879?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/421/0007149880?sid=100: Message: 

Error fetching comments for URL https://n.news

Error fetching comments for URL https://n.news.naver.com/mnews/article/123/0002332988?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/005/0001692261?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/029/0002869601?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/449/0000273384?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/029/0002864599?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/449/0000273288?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/656/0000087657?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/005/0001690484?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/029/0002868950?sid=100: Message: 

Error fetching comments for URL https://n.news

Error fetching comments for URL https://n.news.naver.com/mnews/article/003/0012561321?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/003/0012564329?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/003/0012564192?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/003/0012564195?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/003/0012564193?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/003/0012564186?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/003/0012564194?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/056/0011727147?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/421/0007558874?sid=100: Message: 

Error fetching comments for URL https://n.news

Error fetching comments for URL https://n.news.naver.com/mnews/article/029/0002875635?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/055/0001158387?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/018/0005745786?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/469/0000802982?sid=101: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/030/0003206998?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/003/0012564370?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/003/0012564376?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/003/0012564383?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/003/0012564284?sid=100: Message: 

Error fetching comments for URL https://n.news

Error fetching comments for URL https://n.news.naver.com/mnews/article/123/0002334400?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/052/0002037521?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/003/0012534490?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/656/0000090534?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/003/0012552280?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/449/0000275592?sid=115: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/003/0012552279?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/003/0012552191?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/003/0012552180?sid=100: Message: 

Error fetching comments for URL https://n.news

Error fetching comments for URL https://n.news.naver.com/mnews/article/032/0003292092?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/001/0014645905?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/055/0001149109?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/055/0001149089?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/422/0000656316?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/055/0001159355?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/056/0011731586?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/087/0001047500?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/657/0000026357?sid=100: Message: 

Error fetching comments for URL https://n.news

Error fetching comments for URL https://n.news.naver.com/mnews/article/055/0001132390?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/586/0000073071?sid=102: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/422/0000650564?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/029/0002864695?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/005/0001685137?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/055/0001131423?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/003/0012463731?sid=165: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/018/0005701495?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/016/0002314502?sid=101: Message: 

Error fetching comments for URL https://n.news

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=125.0.6422.113)


In [8]:
#링크, 제목, 본문, 댓글을 데이터로 저장
df_append = pd.DataFrame(results, columns=['url', 'title', 'agency', 'content', 'comment'])
df_append

,url,title,agency,content,comment
0,https://n.news.naver.com/mnews/article/001/001...,"尹대통령, 시에라리온 대통령과 회담…무역 투자 MOU 체결","연합뉴스. All rights reserved. 무단 전재-재배포, AI 학습 및 ...",'K-라이스벨트' 사업 통해 한국 벼 종자 시에라리온에 보급\n\n\n윤석열 대통령...,"[또 호구잡혔네 ㅋ, 부끄럽게 자꾸 여기 저기 얼굴 내밀고 다니지말고 김건희 국정농..."
1,https://n.news.naver.com/mnews/article/023/000...,"尹대통령, 시에라리온 대통령과 회담... 한·아프리카 협력 구체화",조선일보. All rights reserved. 무단 전재 및 재배포 금지.,윤석열 대통령이 31일 용산 대통령실 청사에서 한-아프리카 정상회의 참석을 위해 방...,[이제 현실을 받아드려야한다. 우리 이제 망해서 아프리카 국가와 같은 급의 나라로 ...
2,https://n.news.naver.com/mnews/article/366/000...,"尹 대통령, 내달 4~5일 ‘한·아프리카 정상회의’ 주재... 48개국 대표 참석",조선비즈. All rights reserved. 무단 전재 및 재배포 금지.,25곳 국가원수 방한... “최대 규모 다자정상회담”\n\n윤석열 대통령이 오는 6...,[윤대통령은 이번 아프리카 정상회담 좋은 성과 내시고 건희여사와 아프리카 순방 외교...
3,https://n.news.naver.com/mnews/article/629/000...,"尹 대통령, '자원 부국' 아프리카 25개국 정상과 연쇄 회담한다",더팩트. All rights reserved. 무단 전재 및 재배포 금지.,"31일부터 내달 5일까지 연쇄 정상회담 개최 \n""정부 출범 이래 최대 규모 다자...","[훠훠 중국의 일대일로와 함께하는 윤두창의 중국몽 시즌2 시작인가?, 제발 아무 것..."
4,https://n.news.naver.com/mnews/article/277/000...,"한·아프리카정상회의 48개국 참석…""尹정부 최대 다자 정상회담""",아시아경제. All rights reserved. 무단 전재 및 재배포 금지.,"尹대통령, 25개국과 정상회담\n""아프리카 협력 선택 아닌 필수""\n김태효 국가안보...",[가만히 있으면 아무 일도 안 일어난다.]
...,...,...,...,...,...
2912,https://n.news.naver.com/mnews/article/018/000...,"尹, 과거 수사했던 ‘朴최측근’ 정호성 비서관에 발탁",이데일리. All rights reserved. 무단 전재 및 재배포 금지.,시민사회수석실 3비서관 내정\n박근혜 정부 시절 ‘문고리 3인방’ 불려\n[이데일리...,[허거억 ㅋㅋㅋ 문고리 3인방 비서관 발탁해 ㅋㅋㅋ ㅋㅋㅋ 정신줄 놨구나 니들은 ㅋ...
2913,https://n.news.naver.com/mnews/article/421/000...,"尹 ""5000만에서 80억 시장으로 도전…기업하기 좋은 환경 만들 것""(종합2보)","뉴스1. All rights reserved. 무단 전재 및 재배포, AI학습 이용...",3년 연속 대통령실 잔디마당서 중소기업인 대회 개최\n이재용·구광모·장인화·김동관·...,[네이버 라인도 못지키는 술꾼이 또 취해서 주정부리네. 야당은 잔말말고 빨리 끌어내...
2914,https://n.news.naver.com/mnews/article/119/000...,"윤 대통령, '朴 최측근' 정호성 비서관으로 발탁",데일리안. All rights reserved. 무단 전재 및 재배포 금지.,"尹, 서울중앙지검장 시절\n정호성 관련 사건 수사 지휘\n지난 2022년 특별사면하...","[이게 정상적인 사람인가요?, 비정상의 정상화가 시급함., ㅎㅎ 국민을 바보로봄, ..."
2915,https://n.news.naver.com/mnews/article/366/000...,"尹대통령이 수사했던 ‘朴 최측근’ 정호성, 시민사회수석실 비서관 발탁",조선비즈. All rights reserved. 무단 전재 및 재배포 금지.,윤석열 대통령이 과거 박근혜 정부 시절 '문고리 3인방' 중 한 명으로 꼽혔던 정호...,[평생 검사질만 하던 놈을 뽑아놓으니 인재풀이라고 해봐야 지가 부려먹던 검사와 지가...


In [9]:
#기존 저장 해 놓은 데이터를 불러옴
df_comments = pd.read_csv('news_data_이.csv')
df_comments

,Unnamed: 0,url,title,agency,content,comment
0,0,https://n.news.naver.com/mnews/article/079/000...,"尹, UAE 대통령과 '경제협력' 강화…'새로운 중동붐' 모멘텀",노컷뉴스. All rights reserved. 무단 전재 및 재배포 금지.,"尹대통령, 첫 방한 UAE 대통령과 정상회담\n아랍 국가 최초로 포괄적경제동반자협정...","['이번 계약서 전문 공개해라, 쥐박이 처럼 이면 계약으로 유사시 국군파병 조약 쳐..."
1,1,https://n.news.naver.com/mnews/article/014/000...,"尹대통령, UAE 대통령 최고 예우",파이낸셜뉴스. All rights reserved. 무단 전재 및 재배포 금지.,첫 국빈방문… 전투기 호위\n창덕궁 함께 산책하며 환담\n윤석열 대통령이 첫 국빈 ...,"['국빈방문을 환영합니다~ 좋은 성과 많이 이뤄주세요~', '김치찌개에 소주 대접해..."
2,2,https://n.news.naver.com/mnews/article/001/001...,"앞치마 두른 尹대통령, 기자들과 용산서 김치찌개 만찬","연합뉴스. All rights reserved. 무단 전재-재배포, AI 학습 및 ...","당선인 시절 ""용산 청사 가면 김치찌개 끓여주겠다"" 약속 2년여만에 지켜\n참모진과...","['나라는 쳐 박아 놓고 웃고 있네 와..', '놀고 처먹으려고 대통령 하는 sk...."
3,3,https://n.news.naver.com/mnews/article/023/000...,"앞치마 두른 尹 대통령, 용산서 기자들과 김치찌개 만찬",조선일보. All rights reserved. 무단 전재 및 재배포 금지.,만찬 음식 배식하는 윤석열 대통령. /뉴시스\n\n윤석열 대통령은 24일 대통령실 ...,"['언론의 임무를 다하는 워치독은 없고 펫독만 있네 .....', '사람이라면 과연..."
4,4,https://n.news.naver.com/mnews/article/421/000...,與 지지율 상승인데 尹은 최저치 경신…당정 엇박자 이유는 [여론풍향계],"뉴스1. All rights reserved. 무단 전재 및 재배포, AI학습 이용...","尹 지지율 30% 턱걸이…국힘은 민주 제친 35.2%\n보수 핵심 지지층, 尹대통령...","['윤석열과 빠른 손절이 필요하다고 생각하시는분은 👍 눌러주세요', '한동훈화이팅'..."
...,...,...,...,...,...,...
316,316,https://n.news.naver.com/mnews/article/006/000...,"尹대통령, 기자회견 피하고 또 국무회의? ""변화 의지 없다""",미디어오늘. All rights reserved. 무단 전재 및 재배포 금지.,"총선 패배에도 일방향 소통 고수...“실질적, 실천적 반성 위해서는 기자회견 해야”...","['결국 꼬리말고 도망가는구만~ 쫄보, 모지리 ㅋㅋㅋㅋㅋ 이런것도 지도자라고 30%..."
317,317,https://n.news.naver.com/mnews/article/081/000...,"“尹 대통령 지지율, 32.6%…지난해 10월 이후 최저”",서울신문. All rights reserved. 무단 전재 및 재배포 금지.,"“총선 이틀 뒤 12일 28.2%…일간 최저치 경신”\n“국민의힘 33.6%, 민주...","['구라미터에서 32%면....실제로는 10%대가 맞겠네 ㅋㅋㅋㅋㅋ', '민주당하고..."
318,318,https://n.news.naver.com/mnews/article/002/000...,"尹대통령 지지율, 일간 기준 총선 직후 20%대로 폭락[여론조사 동향]",프레시안. All rights reserved. 무단 전재 및 재배포 금지.,윤석열 대통령의 지지율이 일간 기준으로 총선 직후 20%대를 기록했다는 리얼미터 여...,['아직도 20%대라니 아직멀었다 윤깡통 10%대 가자 역대최악의 무능정권 불통정권...
319,319,https://n.news.naver.com/mnews/article/029/000...,"尹대통령 지지율 4.7%p 하락한 32.6%, 일간 최저치 28.2% 경신",디지털타임스. All rights reserved. 무단 전재 및 재배포 금지.,리얼미터 여론조사\n\n윤석열 대통령 4월 2주차 국정수행 평가. 리얼미터 제공\n...,[]


In [10]:
#이전 데이터와 생성하여 저장한 데이터를 합쳐 새로운 파일로 저장함
df_combined = pd.concat([df_comments, df_append], ignore_index=True)
df_combined.to_csv('news_data_이.csv', encoding = 'utf-8')

In [10]:
data = pd.read_csv('news_data_이.csv', encoding = 'utf-8')
data

,Unnamed: 0.11,Unnamed: 0.10,Unnamed: 0.9,Unnamed: 0.8,Unnamed: 0.7,Unnamed: 0.5,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,Unnamed: 0.6,url,title,agency,content,comment
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,https://n.news.naver.com/mnews/article/029/000...,`3전4기` 끝 성공한 의대 증원…이명박·문재인 정부 번번이 실패,디지털타임스,"의대 증원, 1998년이 마지막\n\n의약분업 때 줄어든 정원, 19년간 '동결'\...",['밥그릇 챙기려고 환자 목숨 볼모로 정부와 국민 협박하는 의사 집단에는 여야 가리...
1,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,https://n.news.naver.com/mnews/article/003/001...,"여 지도부, 이명박 예방…이 ""어려운 시기에 잘 단합해야""(종합)",뉴시스,"황우여·추경호, 연달아 MB 사저 방문해 면담\nMB ""정부와 사전 조율·일치된 모...",[]
2,2,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,https://n.news.naver.com/mnews/article/421/000...,"""노무현·이명박도 특검 받아들였다""…민주, 與 이탈표 챙기기",뉴스1,"尹'10번째 거부권' 행사…野 단독처리·특검추천 등 4가지 이유\n민주 ""과거 특검...",['정부가 잘못한건 제대로 수사하고 고치도록해야하는데 숨기기만 급급하니 어느새 특검...
3,3,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,https://n.news.naver.com/mnews/article/014/000...,"국힘 황우여·추경호, 내일 이명박 전 대통령 예방",파이낸셜뉴스,비대위 회의 입장하는 황우여 비대위원장과 추경호 원내대표 (서울=연합뉴스) 김주형 ...,[]
4,4,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,https://n.news.naver.com/mnews/article/052/000...,"황우여 만난 이명박 ""일치된 여당다운 모습 보여야""",YTN,국민의힘 황우여 비상대책위원장을 만난 이명박 전 대통령은 정부와 사전조율을 통해 일...,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7176,7176,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://n.news.naver.com/mnews/article/001/000...,국민의례하는 이 대통령,"연합뉴스. All rights reserved. 무단 전재-재배포, AI 학습 및 ...",(서울=연합뉴스) 진성철 기자 = 이명박 대통령이 2일 서울 강남구 논현동 건설회관...,[]
7177,7177,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://n.news.naver.com/mnews/article/057/000...,"[시정연설②] 이 대통령, 지방행정개편 협조 당부","MBN. All rights reserved. 무단 전재, 재배포 및 AI학습 이용...",【 앵커멘트 】\n이명박 대통령은 시정연설에서 지방행정체제 개편과 4대 강 살리기에...,[]
7178,7178,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://n.news.naver.com/mnews/article/019/000...,"[시정연설②] 이 대통령, 지방행정개편 협조 당부",MBN. All rights reserved. 무단 전재 및 재배포 금지.,【 앵커멘트 】\n\n이명박 대통령은 시정연설에서 지방행정체제 개편과 4대 강 살리...,[]
7179,7179,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://n.news.naver.com/mnews/article/055/000...,이 대통령 '아세안＋3'정상회의 참석차 태국행,"SBS. All rights reserved. 무단 전재, 재배포 및 AI학습 이용 금지",동남아 순방에 나선 이명박 대통령은 23일 훈센 캄보디아 총리의 안내로 캄보디아 문...,[]


In [11]:
#필요없는 열을 삭제
data = data.drop(['Unnamed: 0.11','Unnamed: 0.10','Unnamed: 0.9','Unnamed: 0.8','Unnamed: 0.7','Unnamed: 0.6','Unnamed: 0.5','Unnamed: 0.4','Unnamed: 0.3','Unnamed: 0.2','Unnamed: 0.1','Unnamed: 0'], axis =1)
data

,url,title,agency,content,comment
0,https://n.news.naver.com/mnews/article/029/000...,`3전4기` 끝 성공한 의대 증원…이명박·문재인 정부 번번이 실패,디지털타임스,"의대 증원, 1998년이 마지막\n\n의약분업 때 줄어든 정원, 19년간 '동결'\...",['밥그릇 챙기려고 환자 목숨 볼모로 정부와 국민 협박하는 의사 집단에는 여야 가리...
1,https://n.news.naver.com/mnews/article/003/001...,"여 지도부, 이명박 예방…이 ""어려운 시기에 잘 단합해야""(종합)",뉴시스,"황우여·추경호, 연달아 MB 사저 방문해 면담\nMB ""정부와 사전 조율·일치된 모...",[]
2,https://n.news.naver.com/mnews/article/421/000...,"""노무현·이명박도 특검 받아들였다""…민주, 與 이탈표 챙기기",뉴스1,"尹'10번째 거부권' 행사…野 단독처리·특검추천 등 4가지 이유\n민주 ""과거 특검...",['정부가 잘못한건 제대로 수사하고 고치도록해야하는데 숨기기만 급급하니 어느새 특검...
3,https://n.news.naver.com/mnews/article/014/000...,"국힘 황우여·추경호, 내일 이명박 전 대통령 예방",파이낸셜뉴스,비대위 회의 입장하는 황우여 비대위원장과 추경호 원내대표 (서울=연합뉴스) 김주형 ...,[]
4,https://n.news.naver.com/mnews/article/052/000...,"황우여 만난 이명박 ""일치된 여당다운 모습 보여야""",YTN,국민의힘 황우여 비상대책위원장을 만난 이명박 전 대통령은 정부와 사전조율을 통해 일...,[]
...,...,...,...,...,...
7176,https://n.news.naver.com/mnews/article/001/000...,국민의례하는 이 대통령,"연합뉴스. All rights reserved. 무단 전재-재배포, AI 학습 및 ...",(서울=연합뉴스) 진성철 기자 = 이명박 대통령이 2일 서울 강남구 논현동 건설회관...,[]
7177,https://n.news.naver.com/mnews/article/057/000...,"[시정연설②] 이 대통령, 지방행정개편 협조 당부","MBN. All rights reserved. 무단 전재, 재배포 및 AI학습 이용...",【 앵커멘트 】\n이명박 대통령은 시정연설에서 지방행정체제 개편과 4대 강 살리기에...,[]
7178,https://n.news.naver.com/mnews/article/019/000...,"[시정연설②] 이 대통령, 지방행정개편 협조 당부",MBN. All rights reserved. 무단 전재 및 재배포 금지.,【 앵커멘트 】\n\n이명박 대통령은 시정연설에서 지방행정체제 개편과 4대 강 살리...,[]
7179,https://n.news.naver.com/mnews/article/055/000...,이 대통령 '아세안＋3'정상회의 참석차 태국행,"SBS. All rights reserved. 무단 전재, 재배포 및 AI학습 이용 금지",동남아 순방에 나선 이명박 대통령은 23일 훈센 캄보디아 총리의 안내로 캄보디아 문...,[]


In [12]:
%%time
print('전처리 전 학습용 샘플의 개수 :',len(data))

# document 열에서 중복인 내용이 있다면 중복 제거
data.drop_duplicates(subset = ['content'], inplace=True) 

# train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 정규 표현식 수행
# 공백은 empty 값으로 변경
data['content'] = data['content'].str.replace('^ +', "")

# document column이 비어있다면(공백), Null 값으로 변경
data['content'].replace('', np.nan, inplace=True)

# Null 값 제거
data = data.dropna(how='any')

#언론사만 나오도록 바꾸기
data['agency'] = data['agency'].str.split('.').str[0]

# index reset
data = data.reset_index(drop=True)
print('전처리 후 학습용 샘플의 개수 :',len(data))

전처리 전 학습용 샘플의 개수 : 7181
전처리 후 학습용 샘플의 개수 : 4828
Wall time: 81.1 ms


<timed exec>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
data.to_csv('news_data_이_final.csv', encoding = 'utf-8')

## 노무현 데이터 수집

In [ ]:
def remove_tag(my_str):
    ## 태그를 지우는 함수
    p = re.compile('(<([^>]+)>)')
    return p.sub('', my_str)

In [ ]:
client_id =
client_secret = 

#네이버 뉴스에서 데이터를 수집
base_url = 'https://openapi.naver.com/v1/search/news.json'

#여러 개의 쿼리를 주어 한번에 여러개의 데이터를 수집
queries = [
    '"盧대통령" +2003', '"盧대통령" +2004','"盧대통령" +2005','"盧대통령" +2006','"盧대통령" +2007',
    '"노대통령" +2003', '"노대통령" +2004', '"노대통령" +2005', '"노대통령" +2006','"노대통령" +2007'
]
n_display = 100

#정확도 순으로 탐색
sort = 'sim'

In [ ]:
# 네이버 API 설정
all_links = []
all_pubdates = []

# HTML 태그 제거 함수
def remove_tag(text):
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

# 반복문을 사용하여 여러 내용을 수집
for query in queries:
    encQuery = urllib.parse.quote(query)
    for start in range(1, 1001, 100):
        url = f'{base_url}?query={encQuery}&display={n_display}&start={start}&sort={sort}'
        my_request = urllib.request.Request(url)
        my_request.add_header("X-Naver-Client-Id", client_id)
        my_request.add_header("X-Naver-Client-Secret", client_secret)
        
        try:
            response = urllib.request.urlopen(my_request)
            rescode = response.getcode()
            if rescode == 200:
                response_body = response.read()
            else:
                print("Error Code:" + rescode)
                continue
        except urllib.error.HTTPError as e:
            print(f"HTTP Error: {e.code} - {e.reason}")
            continue
        except urllib.error.URLError as e:
            print(f"URL Error: {e.reason}")
            continue

        search_results = response_body.decode('utf-8')
        search_results = json.loads(search_results)
        
        if 'items' not in search_results:
            print("No items in search results")
            continue

        p = re.compile('n.news.naver.com')
        for item in search_results['items']:
            if p.search(item['link']) is not None:
                title = html.unescape(remove_tag(item['title']))
                pubdate = item['pubDate']
                link = item['link'].replace('\\', '')
                all_links.append(link)
                all_pubdates.append(pubdate)
        
        time.sleep(1)  # API 호출 제한 방지를 위한 대기 시간

df_link = pd.DataFrame({'link': all_links, 'pubdate': all_pubdates})

In [ ]:
results = []

# df_link의 각 URL에 대해 반복 실행
for i, url in enumerate(df_link['link']):
    driver.get(url)
    
    # 제목을 얻기
    try:
        title = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="title_area"]/span'))
        ).text
    except Exception as e:
        print(f"Error fetching title for URL {url}: {e}")
        title = ""

    # 본문을 얻기
    try:
        content = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="newsct_article"]'))
        ).text
    except Exception as e:
        print(f"Error fetching content for URL {url}: {e}")
        content = ""

    # 언론사 정보를 얻기
    try:
        copyright_text = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, '//p[@class="c_text"]'))
        ).text
        if "ⓒ" in copyright_text:
            agency = copyright_text.split("ⓒ")[1].strip()
        else:
            agency = ""
    except Exception as e:
        print(f"Error fetching agency for URL {url}: {e}")
        agency = ""
    
    # 댓글을 얻기
    try:
        # 전체 댓글 보기 버튼을 클릭
        all_comment_btn = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.CLASS_NAME, 'u_cbox_in_view_comment'))
        )
        all_comment_btn.click()
        
        comments = []
        while True:
            try:
                # 더 많은 댓글 보기 버튼을 클릭
                more_comment_btn = WebDriverWait(driver, 5).until(
                    EC.element_to_be_clickable((By.CLASS_NAME, 'u_cbox_btn_more'))
                )
                more_comment_btn.click()
                time.sleep(1)
            except:
                break
        
        # 댓글 요소를 찾고 텍스트를 저장
        comments_elements = driver.find_elements(By.CLASS_NAME, 'u_cbox_contents')
        comments = [x.text for x in comments_elements]
    except Exception as e:
        print(f"Error fetching comments for URL {url}: {e}")
        comments = []

    time.sleep(1)
    results.append([url, title, agency, content, comments])

# 드라이버 종료
driver.quit()

In [ ]:
#링크, 제목, 본문, 댓글을 데이터로 저장
df_append = pd.DataFrame(results, columns=['url', 'title', 'agency', 'content', 'comment'])
df_append

In [ ]:
#기존 저장 해 놓은 데이터를 불러옴
df_comments = pd.read_csv('news_data_노.csv')
df_comments

In [ ]:
#이전 데이터와 생성하여 저장한 데이터를 합쳐 새로운 파일로 저장함
df_combined = pd.concat([df_comments, df_append], ignore_index=True)
df_combined.to_csv('news_data_노.csv', encoding = 'utf-8')

In [4]:
data = pd.read_csv('news_data_노.csv', encoding = 'utf-8')
data

,Unnamed: 0.1,Unnamed: 0,url,title,content,comment,agency
0,0,0.0,NaN,NaN,NaN,NaN,NaN
1,1,1.0,NaN,NaN,NaN,NaN,NaN
2,2,2.0,NaN,NaN,NaN,NaN,NaN
3,3,3.0,NaN,NaN,NaN,NaN,NaN
4,4,4.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
7762,7762,NaN,https://n.news.naver.com/mnews/article/047/000...,"노 대통령, 싱가포르에 S프로젝트 유치 친서 전달","[오마이뉴스 김당 기자] 노무현 대통령이 18일 신년연설에서 ""정부가 서남해안 개발...",[],오마이뉴스. All rights reserved. 무단 전재 및 재배포 금지.
7763,7763,NaN,https://n.news.naver.com/mnews/article/001/000...,"<노대통령 ""임기내 성과 연연않을 것"">(종합)",(서울=연합뉴스) 김재현 기자 = 노무현(盧武鉉) 대통령의 18일 신년연설에는 그 ...,[],"연합뉴스. All rights reserved. 무단 전재-재배포, AI 학습 및 ..."
7764,7764,NaN,https://n.news.naver.com/mnews/article/014/000...,[노대통령 올 화두는 일자리 창출]“대화·타협으로 국민합의 이뤄야”,노무현 대통령의 18일 신년 대국민연설은 우리사회의 현재와 미래 문제를 대승적 차원...,[],파이낸셜뉴스. All rights reserved. 무단 전재 및 재배포 금지.
7765,7765,NaN,https://n.news.naver.com/mnews/article/028/000...,‘정당개혁’ 칼 뽑아든 노대통령,[한겨레] 노무현 대통령이 ‘유령 당원’을 향해 칼을 뽑아들었다. 정당개혁 없이는 ...,[],"한겨레. All rights reserved. 무단 전재, 재배포 및 크롤링 금지."


In [5]:
#필요없는 열을 삭제
data = data.drop(['Unnamed: 0.1','Unnamed: 0'], axis =1)
data

,url,title,content,comment,agency
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
7762,https://n.news.naver.com/mnews/article/047/000...,"노 대통령, 싱가포르에 S프로젝트 유치 친서 전달","[오마이뉴스 김당 기자] 노무현 대통령이 18일 신년연설에서 ""정부가 서남해안 개발...",[],오마이뉴스. All rights reserved. 무단 전재 및 재배포 금지.
7763,https://n.news.naver.com/mnews/article/001/000...,"<노대통령 ""임기내 성과 연연않을 것"">(종합)",(서울=연합뉴스) 김재현 기자 = 노무현(盧武鉉) 대통령의 18일 신년연설에는 그 ...,[],"연합뉴스. All rights reserved. 무단 전재-재배포, AI 학습 및 ..."
7764,https://n.news.naver.com/mnews/article/014/000...,[노대통령 올 화두는 일자리 창출]“대화·타협으로 국민합의 이뤄야”,노무현 대통령의 18일 신년 대국민연설은 우리사회의 현재와 미래 문제를 대승적 차원...,[],파이낸셜뉴스. All rights reserved. 무단 전재 및 재배포 금지.
7765,https://n.news.naver.com/mnews/article/028/000...,‘정당개혁’ 칼 뽑아든 노대통령,[한겨레] 노무현 대통령이 ‘유령 당원’을 향해 칼을 뽑아들었다. 정당개혁 없이는 ...,[],"한겨레. All rights reserved. 무단 전재, 재배포 및 크롤링 금지."


In [8]:
%%time
print('전처리 전 학습용 샘플의 개수 :',len(data))

# document 열에서 중복인 내용이 있다면 중복 제거
data.drop_duplicates(subset = ['content'], inplace=True) 

# train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 정규 표현식 수행
# 공백은 empty 값으로 변경
data['content'] = data['content'].str.replace('^ +', "")

# document column이 비어있다면(공백), Null 값으로 변경
data['content'].replace('', np.nan, inplace=True)

# Null 값 제거
data = data.dropna(how='any')

#언론사만 나오도록 바꾸기
data['agency'] = data['agency'].str.split('.').str[0]

# index reset
data = data.reset_index(drop=True)
print('전처리 후 학습용 샘플의 개수 :',len(data))

전처리 전 학습용 샘플의 개수 : 5128
전처리 후 학습용 샘플의 개수 : 5031
Wall time: 110 ms


<timed exec>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
data.to_csv('news_data_노_final.csv', encoding = 'utf-8')